# Código necessário para a Prova 01
Aluno: Keanu Frota Sales (202104940028)

#### Imports Necessários

In [1]:
from math import sqrt, log, ceil, erf, exp
from collections.abc import Callable
from csv import reader

#### Função de abertura do arquivo
Note que essa função já retorna uma lista ordenada dos valores através da função sorted

In [2]:
def read_file(file_path: str) -> list[float]:
  with open(file_path, encoding = "utf-8") as file:
    return sorted(float(value) for row in reader(file) for value in row)

#### Funções de Estatística Descritiva
Abaixo estão as funções que calculam, respectivamente, a média, a moda, a mediana, os quartis, a amplitude, a variância, o desvio padrão e o coeficiente de variação dos valores fornecidos.

In [3]:
def mean(values: list[float]) -> float:
  if not values: return 0.0
  return sum(values) / len(values)

def mode(values: list[float]) -> float:
  if not values: return 0.0
  return max(set(values), key = values.count)

def median(values: list[float]) -> float:
  if not values: return 0.0
  div, mod = divmod(len(values), 2)
  return (values[div + mod - 1] + values[div]) / 2

def quartis(values: list[float]) -> tuple[float, float, float]:
  if not values: return 0.0, 0.0, 0.0
  div, mod = divmod(len(values), 2)
  q1 = median(values[:div])
  q2 = (values[div + mod - 1] + values[div]) / 2
  q3 = median(values[div + mod:])
  return (q1, q2, q3)

def amplitude(values: list[float]) -> float:
  if not values: return 0.0
  return max(values) - min(values)

def variance(values: list[float]) -> float:
  if not values: return 0.0
  mean_value = mean(values)
  sums = sum((value - mean_value) ** 2 for value in values)
  return sums / len(values)

def default_deviation(values: list[float]) -> float:
  if not values: return 0.0
  return sqrt(variance(values))

def variation_coefitient(values: list[float]) -> float:
  mean_value = mean(values)
  if not values or not mean_value: return 0.0
  return 100 * default_deviation(values) / mean_value

#### Funções de Detecção/Remoção de Outliers
Note que é possivel detectar/remover tanto outliers moderados quanto extremos

In [4]:
def detect_outliers(values: list[float], extreme: bool = False) -> list[float]:
  if not values: return []
  q1, _, q3 = quartis(values)
  _3_6_halfs_iqr = 1.5 * (q3 - q1) * (1 + extreme)
  lower, upper = q1 - _3_6_halfs_iqr, q3 + _3_6_halfs_iqr
  return [value for value in values if value < lower or value > upper]

def remove_outliers(values: list[float], extreme: bool = False) -> list[float]:
  if not values: return []
  q1, _, q3 = quartis(values)
  _3_6_halfs_iqr = 1.5 * (q3 - q1) * (1 + extreme)
  lower, upper = q1 - _3_6_halfs_iqr, q3 + _3_6_halfs_iqr
  return [value for value in values if lower <= value <= upper]

#### Funções de Criação e Plotagem do histograma
Note que aqui a quantidade de classes aqui é definida como ceil(3.322 * log(len(values), 10) + 1).

In [5]:
def histogram(values: list[float]) -> list[int]:
  if not values: return []
  bins = ceil(3.322 * log(len(values), 10) + 1)
  min_value, max_value = min(values), max(values)
  bin_size = (max_value - min_value) / bins
  histogram_data = [0] * bins
  for value in values:
    index = int((value - min_value) / bin_size)
    if index >= bins: index = bins - 1
    histogram_data[index] += 1
  return histogram_data

def plot_histogram(histogram_data: list[int]) -> None:
  if not histogram_data: return
  max_value = max(histogram_data)
  length = len(histogram_data)
  print(f"\nNúmero de classes: {length}")
  print("Histograma dos valores:")
  for i, count in enumerate(histogram_data):
    bar = "*" * (count * 50 // max_value)
    print(f"{i + 1}: {bar} (freq.: {count})")

#### Funções do Teste de Kolmogorov Smirnov (KS)

Primeiramente, temos as definições das funções de distribuição acumulada conhecidas, que são, respectivamente, a uniforme, a exponencial, a normal, a lognormal e a triangular. Observe que no teste KS **devemos** usar as funções de distribuição **acumulada**, não as funções de distribuição de probablidade.

Após essas definições, essas funções são usadas dentro do teste Kolmogorov Smirnov para saber se aquela distribuição é aderente ou não ao conjunto de dados

In [6]:
def uniform(values: list[float]) -> Callable[[float], float]:
  if not values: return lambda x: 0.0
  diff = (b := max(values)) - (a := min(values))
  return lambda x: 0.0 if x < a else (x - a) / diff if x < b else 1.0

def exponential(values: list[float]) -> Callable[[float], float]:
  if not values: return lambda x: 0.0
  scale = len(values) / sum(values)
  return lambda x: 0.0 if x < 0 else 1 - exp(-scale * x)

def normal(values: list[float]) -> Callable[[float], float]:
  if not values: return lambda x: 0.0
  mu = sum(values) / (len_values := len(values))
  sigma = sqrt(2 * sum((v - mu) ** 2 for v in values) / len_values)
  return lambda x: 0.5 * (1 + erf((x - mu) / sigma))

def lognormal(values: list[float]) -> Callable[[float], float]:
  positive = [v for v in values if v > 0]
  if not values or not positive: return lambda x: 0.0
  mu = sum(log(v) for v in positive) / (len_positive := len(positive))
  sigma = sqrt(2 * sum((log(v) - mu) ** 2 for v in positive) / len_positive)
  return lambda x: 0.0 if x <= 0 else 0.5 * (1 + erf((log(x) - mu) / sigma))

def triangular(values: list[float]) -> Callable[[float], float]:
  if not values: return lambda x: 0.0
  a, b, c = min(values), mode(values), max(values)
  lower, upper = (c - a) * (b - a), (c - a) * (c - b)
  return lambda x: (0.0 if x < a else
    ((x - a) ** 2) / lower if x < b else
    1 - ((c - x) ** 2) / upper if x < c else 1.0)

KSFunction = Callable[[list[float]], Callable[[float], float]]

def kolmogorov_smirnov(values: list[float], distribution: KSFunction) -> bool:
  if not values: return False
  distrib_name = distribution.__name__.capitalize()
  try:
    n = len(values)
    empirical = [(i + 1) / n for i in range(n)]
    function = distribution(values)
    teorically = [function(x) for x in values]
  except ZeroDivisionError:
    print(f"Erro: Divisão por zero na distribuição \"{distrib_name}\".")
    return False
  d = max(abs(e - t) for e, t in zip(empirical, teorically))
  critical_value = 1.36 / sqrt(n)
  print(f"D {distrib_name}: {d}, Valor crítico: {critical_value}")
  return d < critical_value

# Resolução das Questões da Prova 01
Primeiramente, vamos abrir o arquivo fornecido e fazer 2 listas, uma com os valores com outliers e outra sem outliers, ambas ordenadas.

In [7]:
with_outliers = read_file("entrada.txt")
without_outliers = remove_outliers(with_outliers)

1ª) A partir do conjunto de dados utilizado como entrada na prova, realize o tratamento de outliers e forneça os seguintes valores:

a) Quantos outliers existem e quais são os seus valores? (1,0) 

In [8]:
outliers = detect_outliers(with_outliers)
print(f"Os outliers são: {outliers} (Quantidade: {len(outliers)})")

Os outliers são: [276.8772, 331.3165, 454.4438, 4963.7838] (Quantidade: 4)


b) Qual a média? (1,0)

In [9]:
print(f"Media com outliers: {mean(with_outliers)}, sem outliers: {mean(without_outliers)}")

Media com outliers: 28.012854560585883, sem outliers: 26.041397933333332


c) Qual o desvio padrão? (1,0)

In [10]:
print(f"Desvio padrão com outliers: {default_deviation(with_outliers)}")
print(f"Desvio padrão sem outliers: {default_deviation(without_outliers)}")

Desvio padrão com outliers: 90.73868132926775
Desvio padrão sem outliers: 2.8398495700316704


d) Informe a função de probabilidade descrita por estes dados, o número de classes do histograma, e  a frequência de cada classe (2,0).

Vamos por partes. Primeiro, vamos mostrar o histograma que esses dados apresentam, com e sem outliers.

In [11]:
print("Histograma com outliers presentes:")
plot_histogram(histogram(with_outliers))

Histograma com outliers presentes:

Número de classes: 13
Histograma dos valores:
1: ************************************************** (freq.: 3002)
2:  (freq.: 1)
3:  (freq.: 0)
4:  (freq.: 0)
5:  (freq.: 0)
6:  (freq.: 0)
7:  (freq.: 0)
8:  (freq.: 0)
9:  (freq.: 0)
10:  (freq.: 0)
11:  (freq.: 0)
12:  (freq.: 0)
13:  (freq.: 1)


In [12]:
print("Histograma sem outliers presentes:")
plot_histogram(histogram(without_outliers))

Histograma sem outliers presentes:

Número de classes: 13
Histograma dos valores:
1: ************ (freq.: 105)
2: ***************************************** (freq.: 341)
3: ************************************************** (freq.: 414)
4: ************************************************** (freq.: 414)
5: **************************************** (freq.: 336)
6: ************************************ (freq.: 305)
7: ********************************* (freq.: 276)
8: ****************************** (freq.: 252)
9: ********************** (freq.: 184)
10: ******************** (freq.: 169)
11: ************* (freq.: 108)
12: ******** (freq.: 74)
13: ** (freq.: 22)


Como podemos ver nas respostas acima, a presença dos outliers prejudica a criação correta do histograma. Por isso, eles devem ser removidos para que o histograma seja criado da forma correta.

Levando em conta apenas o histograma **sem outliers**, temos um total de 13 classes com as frequências sendo mostradas no próprio histograma.

Fazendo a análise visual do histograma **sem outliers**, podemos induzir que os dados se aproximam da aderência com a distribuição lognormal e a distribuição triangular. Para termos certeza, vamos aplicar o teste de Kolmogorov Smirnov nas 5 distribuições conhecidas com os dados **sem outliers**:

In [13]:
for function in (uniform, exponential, normal, lognormal, triangular):
  distribution = function.__name__.capitalize()
  ks_result = "são" if kolmogorov_smirnov(without_outliers, function) else "não são"
  print(f"Os dados {ks_result} compatíveis com a distribuição {distribution}.\n")

D Uniform: 0.20203079128635382, Valor crítico: 0.024830089273567533
Os dados não são compatíveis com a distribuição Uniform.

D Exponential: 0.5556318816587541, Valor crítico: 0.024830089273567533
Os dados não são compatíveis com a distribuição Exponential.

D Normal: 0.07620804248727908, Valor crítico: 0.024830089273567533
Os dados não são compatíveis com a distribuição Normal.

D Lognormal: 0.06358643303927913, Valor crítico: 0.024830089273567533
Os dados não são compatíveis com a distribuição Lognormal.

D Triangular: 0.06919609997661025, Valor crítico: 0.024830089273567533
Os dados não são compatíveis com a distribuição Triangular.



Aqui vemos que a análise matemática realmente é importante. Apesar de o teste KS mostrar que realmente as distribuições lognormal e triangular são as que mais se aproximam da aderência com o conjunto de dados, elas não conseguem ter um D menor que o D crítico, necessário para se concluir que realmente são aderentes. Portanto, o conjunto de dados **NÃO** é aderente a nenhuma função de distribuição conhecida.